In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from dependencies import *

from experiments import run_experiment
from objectives import *
from datasets import *
from utils import *
from plotting import *

In [21]:
# dataset options
dataset_num = 6
data_dir = './'

is_subsample = 0
is_kernelize = 0
subsampled_n = -1

# set the seed for reproducibility
np.random.seed(6162647)

In [22]:
# optimization options
max_epochs = 20
num_restarts = 1
batch_size = 100
verbose = True

In [23]:
# choose experiment
exp_num = 5

In [24]:
# problem size when generating synthetic data
if dataset_num == -1:
    n, d = 10000, 20
    false_ratio = 0.25
    margin = 1e-6
    print(is_kernelize)
    A, y, A_test, y_test = data_load(data_dir, dataset_num,n, d, margin, false_ratio)
else:
    if is_subsample == 1:
        n = subsampled_n
    else:
        n = 0
    if is_kernelize == 1:
        d = n
    else:
        d = 0
    false_ratio = 0
    
    A, y, A_test, y_test = data_load(data_dir, dataset_num, n,d, false_ratio, is_subsample, is_kernelize)
    
    if n == 0:
        n = A.shape[0]
    
    
#define the regularized losses we will use
logistic_closure_l2 = make_closure(logistic_loss, 1/n)
squared_hinge_closure_l2 = make_closure(squared_hinge_loss, 1/n)
squared_closure_l2 = make_closure(squared_loss, 1/n)

Loaded  ijcnn  dataset.


In [25]:
if exp_num == 0:
    print('Testing SVRG-BB:')
    init_eta = 1e-3
    
    num_variants = 1
    
    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))
    
    x_mean, results_mean[0, :], results_std[0, :], results2_mean[0, :], results2_std[0, :] = run_experiment(svrg_bb, logistic_closure_no_l2, A, y, A_test, y_test, init_eta, batch_size, max_epochs, num_restarts, verbose=verbose)

In [26]:
if exp_num == 1:
    print('Testing robustness to step-size for SVRG:')    
    count = 0
    labels = []
    step_sizes = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
    colors = ['r', 'b', 'g','k','cyan','lightgreen','gray']

    num_variants = len(step_sizes)

    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))

    for init_eta in step_sizes: 
        print('Begin to run SVRG with step-size = :', init_eta)
        x_mean, results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
        run_experiment(method_name = 'svrg', closure = logistic_closure_l2, \
                       X = A, y = y, X_test = A_test, y_test = y_test, \
                       init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                       m =0, num_restarts = num_restarts, verbose=verbose)
        count = count + 1
        labels.append(str(init_eta))
    

In [27]:
if exp_num == 2:
    print('Testing robustness to size of inner loop for SVRG:')   

    count = 0
    labels = []
    init_eta = 1e-2
    inner_loop_list = [0.5 , 1, 2, 5, 10]

    colors = ['r', 'b', 'g','k','cyan','lightgreen','gray']

    num_variants = len(inner_loop_list)


    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))

    for inner_loop in inner_loop_list: 
        print('Begin to run SVRG with inner loop size = :', inner_loop)

        m = int(inner_loop * n)

        x_mean, results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
        run_experiment(method_name = "svrg", closure = logistic_closure_l2, \
                       X = A, y = y, X_test = A_test, y_test = y_test, \
                       init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                       m = m, num_restarts = num_restarts, verbose=verbose)
        count = count + 1
        labels.append(str(inner_loop))

In [28]:
if exp_num == 3:
    
    labels = []
    colors = ['r', 'b', 'g','k','cyan','lightgreen','gray']

    step_sizes = [1e-3]
        
    num_variants = len(step_sizes)

    grad_evals = np.zeros((num_variants, max_epochs))    
    
    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))     
        
    for init_eta in step_sizes:     
  
        print('Testing SVRG-Ada with step-size:', init_eta)
    
        x_mean, grad_evals[0,:], results_mean[0, :], results_std[0, :], results2_mean[0, :], results2_std[0, :] =\
        run_experiment(method_name = "svrg_ada", closure = squared_closure_l2, \
                           X = A, y = y, X_test = A_test, y_test = y_test, \
                           init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                           m = int(n / batch_size), num_restarts = num_restarts, verbose=verbose, linesearch_option = 0, max_sgd_warmup_epochs = 0, adaptive_termination = False) 
    
        labels.append(['SVRG-Ada-', str(init_eta)])


In [29]:
if exp_num == 4:
    print('Testing SVRG vs SVRG-Ada vs SVRG-BB wrt step-size robustness:')

    count = 0
    labels = []
    step_sizes = [1]
    colors = ['r', 'b', 'g','r','b','g','r', 'b', 'g']

    num_variants = 3 * len(step_sizes)
    
    grad_evals = np.zeros((num_variants, max_epochs))
    
    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))
    
    closure = logistic_closure_l2

    for init_eta in step_sizes: 
#         print('Begin to run SVRG with step-size = :', init_eta)

#         # SVRG
#         x_mean, grad_evals[count,:], results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
#         run_experiment(method_name = "svrg", closure = closure, \
#                        X = A, y = y, X_test = A_test, y_test = y_test, \
#                        init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
#                        m = int(n / batch_size), num_restarts = num_restarts, verbose=verbose, adaptive_termination = False)
#         count = count + 1
#         labels.append(str(init_eta))
        
#         print('Begin to run SVRG BB with step-size = :', init_eta)        
#         # SVRG BB
#         x_mean, grad_evals[count,:], results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
#         run_experiment(method_name = "svrg_bb", closure = closure, \
#                        X = A, y = y, X_test = A_test, y_test = y_test, \
#                        init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
#                        m = int(n / batch_size), num_restarts = num_restarts, verbose=verbose, adaptive_termination = False)
#         count = count + 1
#         labels.append('BB' + str(init_eta))
        
        print('Begin to run SVRG Ada with step-size = :', init_eta)
        # Ada SVRG
        x_mean, grad_evals[count,:], results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
        run_experiment(method_name = "svrg_ada", closure = closure, \
                       X = A, y = y, X_test = A_test, y_test = y_test, \
                       init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                       m = int(n / 2 * batch_size), num_restarts = num_restarts, verbose=verbose, linesearch_option = 0, 
                       max_sgd_warmup_epochs = 0, adaptive_termination = False)
        count = count + 1
        labels.append('Ada' + str(init_eta))
    

In [30]:
if exp_num == 5:
    print('Testing SVRG vs SVRG with diagnostic')

    count = 0
    labels = []
    step_sizes = [1e-1]
    colors = ['r', 'b', 'g','r','b','g','r', 'b', 'g']

    num_variants = 3 * len(step_sizes)
    
    grad_evals = np.zeros((num_variants, max_epochs))
    
    results_mean = np.zeros((num_variants, max_epochs))
    results_std = np.zeros((num_variants, max_epochs))

    results2_mean = np.zeros((num_variants, max_epochs))
    results2_std = np.zeros((num_variants, max_epochs))
    
    closure = logistic_closure_l2

    for init_eta in step_sizes: 
        print('Begin to run SVRG with step-size = :', init_eta)

        # SVRG
        x_mean, grad_evals[count,:], results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
        run_experiment(method_name = "svrg", closure = closure, \
                       X = A, y = y, X_test = A_test, y_test = y_test, \
                       init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                       m = int(n / batch_size), num_restarts = num_restarts, verbose=verbose, adaptive_termination = False)
        count = count + 1
        labels.append(str(init_eta))
        
        print('Begin to run SVRG with step-size = :', init_eta)

        # SVRG
        x_mean, grad_evals[count,:], results_mean[count, :], results_std[count, :], results2_mean[count, :], results2_std[count, :] =\
        run_experiment(method_name = "svrg", closure = closure, \
                       X = A, y = y, X_test = A_test, y_test = y_test, \
                       init_step_size = init_eta, batch_size = batch_size, max_epochs = max_epochs, \
                       m = int(n / batch_size), num_restarts = num_restarts, verbose=verbose, adaptive_termination = True)
        count = count + 1
        labels.append('Diagnostic_' + str(init_eta))
    

Testing SVRG vs SVRG with diagnostic
Begin to run SVRG with step-size = : 0.1
Running Experiment:
Epoch.: 0, Grad. norm: 1.60e-01, Func. value: 6.931472e-01, Num gradient evaluations: 0
Epoch.: 1, Grad. norm: 6.11e-02, Func. value: 3.917728e-01, Num gradient evaluations: 56000
Epoch.: 2, Grad. norm: 3.20e-02, Func. value: 3.354554e-01, Num gradient evaluations: 112000
Epoch.: 3, Grad. norm: 2.08e-02, Func. value: 3.167600e-01, Num gradient evaluations: 168000
Epoch.: 4, Grad. norm: 1.58e-02, Func. value: 3.076377e-01, Num gradient evaluations: 224000
Epoch.: 5, Grad. norm: 1.35e-02, Func. value: 3.017407e-01, Num gradient evaluations: 280000
Epoch.: 6, Grad. norm: 1.23e-02, Func. value: 2.971558e-01, Num gradient evaluations: 336000
Epoch.: 7, Grad. norm: 1.16e-02, Func. value: 2.932050e-01, Num gradient evaluations: 392000
Epoch.: 8, Grad. norm: 1.11e-02, Func. value: 2.896193e-01, Num gradient evaluations: 448000
Epoch.: 9, Grad. norm: 1.07e-02, Func. value: 2.862835e-01, Num gradien

KeyboardInterrupt: 

In [ ]:
colors = ['r', 'b', 'g','r','b','g','r', 'b', 'g']
fig = plot_shaded_error_bars(grad_evals = grad_evals, results_mean = results_mean, results_std = results_std, ylabel = 'Loss', max_epochs = max_epochs, colors = colors, labels = labels)
plt.show()

fig = plot_shaded_error_bars(grad_evals = grad_evals, results_mean = results2_mean, results_std = results2_std, ylabel = 'Grad_norm', max_epochs = max_epochs, colors = colors, labels = labels)
plt.show()


